# Data pre-processing

Para tener los datos necesario para probar los enfoques de filtros colaborativos(CF) y basados en contenido(CB) necesito:

* Calificaciones de los ítems(movies) de los usuarios (CF)
* Features propies de los ítems (CB)

Dado esto, a continuacion se combinaran los siguientes datasets:

* MovieLens 25M Dataset: Prácticamente no tiene información de las películas pero si tiene las calificaciones de los usuarios.
* TMDB Movie Dataset: No tiene calificaciones personalizadas como el dataset anterior pero tiene varios features para las películas que es lo que necesito.

## Setup

In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import sys
sys.path.append('../lib')

from database.mongo import Mongo
from pytorch_common.util import LoggerBuilder

In [37]:
LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

# Pre-processing steps

#### 1. Import five collecitons to **movies** mongodb database:

    * From movie lens dataset:
        * rattings
        * movies
        * links
        * tags
    * From the movie database dataset:
        * movies_metadata

In [38]:
DATASETS_PATH   = '../datasets'
MOVIE_LENS_PATH = f'{DATASETS_PATH}/ml-25m'
TMDB_PATH       = f'{DATASETS_PATH}/tmdb'

In [39]:
!mkdir -p {DATASETS_PATH}

In [17]:
!cd {DATASETS_PATH}; wget https://files.grouplens.org/datasets/movielens/ml-25m.zip

--2023-02-04 21:07:50--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip           41%[=======>            ] 102.88M   250KB/s    in 9m 2s   

2023-02-04 21:16:53 (194 KB/s) - Read error at byte 107872256/261978986 (Error decoding the received TLS packet.). Retrying.

--2023-02-04 21:16:54--  (try: 2)  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 261978986 (250M), 154106730 (147M) remaining [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip           97%[++++++++====

In [18]:
!cd {DATASETS_PATH}; unzip ml-25m.zip

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [8]:
!mkdir -p {TMDB_PATH}

In [9]:
!cd {TMDB_PATH}; wget https://huggingface.co/spaces/Kamand/Movie_Recommendation/resolve/main/movies_metadata.csv

--2023-02-06 11:35:07--  https://huggingface.co/spaces/Kamand/Movie_Recommendation/resolve/main/movies_metadata.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving huggingface.co (huggingface.co)... 3.231.67.228, 54.235.118.239, 2600:1f18:147f:e850:e203:c458:10cd:fc3c, ...
Connecting to huggingface.co (huggingface.co)|3.231.67.228|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/17/34/17349aa1da154cac90daa82631575d4a0cbcb43ebae58110821f28039b74bd09/37ec322786b136a24604564e55160be75f8937546f11cb1a990076971fa895fd?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27movies_metadata.csv%3B+filename%3D%22movies_metadata.csv%22%3B&Expires=1675953309&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzE3LzM0LzE3MzQ5YWExZGExNTRjYWM5MGRhYTgyNjMxNTc1ZDRhMGNiY2I0M2ViYWU1ODExMDgyMWYyODAzOWI3NGJkMDkvMzdlYzMyMjc4NmIxMzZhMjQ2MDQ1NjRlNTUxNjBiZTc1Zjg5Mzc1NDZmMTF

In [13]:
MOVIE_LENS_FILES = [ f'{MOVIE_LENS_PATH}/{f}' for f in [ 
    'ratings.csv',
    'movies.csv',
    'links.csv',
    'tags.csv'
]]
TMDB_FILES = [ f'{TMDB_PATH}/movies_metadata.csv']

In [19]:
Mongo.import_csv(database = 'movies', file_paths = TMDB_FILES)

In [17]:
Mongo.import_csv(database = 'movies', file_paths = MOVIE_LENS_FILES)

2023-02-06 11:49:45,179 - ERROR - b'2023-02-06T11:42:11.746-0300\tconnected to: mongodb://localhost/\n2023-02-06T11:42:14.747-0300\t[........................] movies.ratings\t2.60MB/647MB (0.4%)\n2023-02-06T11:42:17.747-0300\t[........................] movies.ratings\t5.60MB/647MB (0.9%)\n2023-02-06T11:42:20.747-0300\t[........................] movies.ratings\t8.82MB/647MB (1.4%)\n2023-02-06T11:42:23.747-0300\t[........................] movies.ratings\t12.0MB/647MB (1.9%)\n2023-02-06T11:42:26.747-0300\t[........................] movies.ratings\t14.7MB/647MB (2.3%)\n2023-02-06T11:42:29.747-0300\t[........................] movies.ratings\t17.6MB/647MB (2.7%)\n2023-02-06T11:42:32.747-0300\t[........................] movies.ratings\t20.8MB/647MB (3.2%)\n2023-02-06T11:42:35.747-0300\t[........................] movies.ratings\t23.9MB/647MB (3.7%)\n2023-02-06T11:42:38.747-0300\t[........................] movies.ratings\t26.5MB/647MB (4.1%)\n2023-02-06T11:42:41.748-0300\t[#....................

#### 2. Transform imdb id to number.

```js
// Mongodb query
db.getCollection('movies_metadata').aggregate([
    {
        $match: { 
            $and: [
                { imdb_id: { $ne: "" } },
                { imdb_id: { $ne: 0 } }
            ]
        }
    },
    {
        $addFields: {
            imdb_id: {$toLong: [ { $arrayElemAt: [ { $split: ["$imdb_id", "tt"]}, 1 ] }] }
        }
    },
    { $out: "movies_metadata_v2" }
]);
```

#### 3.  Add indexes to both links and movies_metadata_v2 collections.

```js
// Mongodb queries
db.getCollection('links').createIndex(
    { 'movieId': 1 }, 
    { unique: true, name: 'movieId_unique_index' }
)
db.getCollection('movies_metadata_v2').createIndex(
    { 'imdb_id': 1 }, 
    { unique: false, name: 'imdb_id_multiple_index' }
)
```

#### 4. Add imdb features to movies collection.

```js
// Mongodb queries
db.getCollection('movies').aggregate([
    {
        $lookup:
          {
            from: "links",
            foreignField: "movieId",
            localField: "movieId", 
            as: "links"
          }
     },
     { $match: { links: { $exists: true, $not: {$size: 0} } } },
     { 
        $project: { 
            id: "$movieId",
            tmdb_id:      { "$arrayElemAt": ["$links.tmdbId", 0] },
            imdb_id:      { "$arrayElemAt": ["$links.imdbId", 0] },
            title:        { $arrayElemAt:   [ {$split:["$title","("]} ,  0 ] },
            release_year: { $arrayElemAt:   [ {$split:["$title","("]} ,  1 ] },
            genres:       { $split:         [ "$genres", "|" ] }
        } 
    }, 
    {
        $lookup:
          {
            from: "movies_metadata_v2",
            foreignField: "imdb_id",
            localField: "imdb_id", 
            as: "movies_metadata"
          }
     },
     { $match: {  movies_metadata: { $exists: true, $not: {$size: 0} } } },
     { 
        $project: { 
            id: 1,
            tmdb_id: 1,
            imdb_id: 1,
            title: 1,
            genres: 1,
            for_adults:        { "$arrayElemAt": ["$movies_metadata.adult", 0] },
            budget:            { "$arrayElemAt": ["$movies_metadata.budget", 0] },
            original_language: { "$arrayElemAt": ["$movies_metadata.original_language", 0] },
            overview:          { "$arrayElemAt": ["$movies_metadata.overview", 0] },
            poster:            { "$arrayElemAt": ["$movies_metadata.poster_path", 0] },
            release:           { "$arrayElemAt": ["$movies_metadata.release_date", 0] },
            popularity:        { "$arrayElemAt": ["$movies_metadata.popularity", 0] },
            vote_mean:         { "$arrayElemAt": ["$movies_metadata.vote_average", 0] },
            vote_count:        { "$arrayElemAt": ["$movies_metadata.vote_count", 0] }
        }
    },
    { $out: "movies_v2" }
]);
```

#### 5. Group tags per used, movie pair.

```js
// Mongodb queries
db.getCollection('tags').aggregate(
    [
        { 
            $group: {
                _id: {
                    user_id: "$userId",
                    movie_id: "$movieId"
                },
                tags: { $push: { $toLower: "$tag" } }
            }
        },
        {
          $project: {
            _id: 0,
            user_id: "$_id.user_id",
            movie_id: "$_id.movie_id",
            user_movie_id: { $concat: [ { $toString: "$_id.user_id" } , "_", { $toString:"$_id.movie_id"} ] },
            tags: 1
          }  
        },
        { $out: "tags_v2" }
    ]
);
```

#### 6. Create used_movie_id into new tags_v2 collection.

```js
// Mongodb queries
db.getCollection('tags_v2').createIndex(
    { 'user_movie_id': 1 }, 
    { unique: true, name: 'id_unique_index' }
)
```

#### 7. Add used_movie_id fields into new ratings_v2 collection and also create a unique index.

```js
// Mongodb queries
db.getCollection('ratings').aggregate([
        {
          $project: {
            user_id: "$userId",
            movie_id: "$movieId",
            user_movie_id: { $concat: [ { $toString: "$userId" } , "_", { $toString:"$movieId"} ] },
            rating: 1,
            timestamp: 1
          }  
        },
        { $out: "ratings_v2" }
    ]
);
db.getCollection('ratings_v2').createIndex(
    { 'user_movie_id': 1 }, 
    { unique: true, name: 'id_unique_index' }
);
```

#### 8. Join ratting_v2 and tags_v2 collections by user_movie_id into a new ratings_tags_v1 collection.

```js
// Mongodb queries
db.getCollection('ratings_v2').aggregate([
    {
        $lookup:
          {
            from: "tags_v2",
            foreignField: "user_movie_id",
            localField: "user_movie_id", 
            as: "tags_v2"
          }
     },
     { $match: { tags_v2: { $exists: true, $not: {$size: 0} } } },
     { 
        $project: { 
            user_id: 1,
            movie_id: 1,
            rating: 1,
            timestamp: 1,
            tags: "$tags_v2.tags"
        }
    },
    {
        $addFields: {  
            _id: { $concat: [ { $toString: "$user_id" } , "_", { $toString:"$movie_id"} ] },            
            tags: {
                "$reduce": {
                    "input": "$tags",
                    "initialValue": [],
                    "in": { "$setUnion": [ "$$value", "$$this" ] }
                }
            }
        }
    },
    { $out: "ratings_tags_v1" }
]);
```

#### 9. Add tags field into movie_v2 collections:

```js
// Mongodb queries
db.getCollection('movies_v2').createIndex(
    { 'id': 1 }, 
    { unique: true, name: 'id_unique_index' }
)
db.getCollection('tags_v2').createIndex(
    { 'movie_id': 1 }, 
    { unique: true, name: 'id_unique_index' }
)
db.getCollection('movies_v2').aggregate([
    {
        $lookup:
          {
            from: "tags_v2",
            foreignField: "movie_id",
            localField: "id",
            as: "tags_v2"
          }
    },
    { $match: { tags_v2: { $exists: true, $not: {$size: 0} } } },
    { 
        $addFields: { 
            tags: {
                "$reduce": {
                    "input": "$tags_v2.tags",
                    "initialValue": [],
                    "in": { "$setUnion": [ "$$value", "$$this" ] }
                }
            }
        }
    },
    { $unset: ["tags_v2"] },
    { $addFields: {  _id: "$id" } },
    { $unset: ["id"] },
    { $out: "movies_v3" }
]);
```

#### 10. Export final ccollections to json files:

* movies_v3 to movies_v3.json
* ratings_tags_v1 to ratings_tags_v1.json

In [40]:
Mongo.export_to_json(database='movies', path=DATASETS_PATH, collections=['movies_v3'])

2023-02-06 17:56:37,777 - ERROR - b'2023-02-06T17:56:35.580-0300\tconnected to: mongodb://localhost/\n2023-02-06T17:56:36.581-0300\t[#####...................]  movies.movies_v3  8000/35149  (22.8%)\n2023-02-06T17:56:37.580-0300\t[################........]  movies.movies_v3  24000/35149  (68.3%)\n2023-02-06T17:56:37.769-0300\t[########################]  movies.movies_v3  35149/35149  (100.0%)\n2023-02-06T17:56:37.769-0300\texported 35149 records\n'
2023-02-06 17:56:37,777 - ERROR - b'2023-02-06T17:56:35.580-0300\tconnected to: mongodb://localhost/\n2023-02-06T17:56:36.581-0300\t[#####...................]  movies.movies_v3  8000/35149  (22.8%)\n2023-02-06T17:56:37.580-0300\t[################........]  movies.movies_v3  24000/35149  (68.3%)\n2023-02-06T17:56:37.769-0300\t[########################]  movies.movies_v3  35149/35149  (100.0%)\n2023-02-06T17:56:37.769-0300\texported 35149 records\n'
2023-02-06 17:56:37,777 - ERROR - b'2023-02-06T17:56:35.580-0300\tconnected to: mongodb://localh

In [41]:
Mongo.export_to_json(database='movies', path=DATASETS_PATH, collections=['ratings_tags_v1'])

2023-02-06 17:56:43,175 - ERROR - b'2023-02-06T17:56:37.816-0300\tconnected to: mongodb://localhost/\n2023-02-06T17:56:38.818-0300\t[###.....................]  movies.ratings_tags_v1  32000/210725  (15.2%)\n2023-02-06T17:56:39.817-0300\t[########................]  movies.ratings_tags_v1  72000/210725  (34.2%)\n2023-02-06T17:56:40.817-0300\t[############............]  movies.ratings_tags_v1  112000/210725  (53.1%)\n2023-02-06T17:56:41.818-0300\t[#################.......]  movies.ratings_tags_v1  152000/210725  (72.1%)\n2023-02-06T17:56:42.818-0300\t[#####################...]  movies.ratings_tags_v1  192000/210725  (91.1%)\n2023-02-06T17:56:43.172-0300\t[########################]  movies.ratings_tags_v1  210725/210725  (100.0%)\n2023-02-06T17:56:43.172-0300\texported 210725 records\n'
2023-02-06 17:56:43,175 - ERROR - b'2023-02-06T17:56:37.816-0300\tconnected to: mongodb://localhost/\n2023-02-06T17:56:38.818-0300\t[###.....................]  movies.ratings_tags_v1  32000/210725  (15.2%)\